# Joins, Aggregations y función Window en profundidad

### Preparación de entorno y carga de datos

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.window import Window

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PySpark02").getOrCreate()

movies = spark.read.csv("../../data/movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv("../../data/ratings.csv", header=True, inferSchema=True)

## JOINs en PySpark

Los joins permiten combinar datos de dos DataFrames en función de columnas comunes (que se recomienda que sean ids). Es fundamental para enriquecer un conjunto de datos con información de otro.

#### Tipos de Joins

|Tipo|Descripción|
|----------|----------|
|inner|Devuelve solo las filas que coinciden en ambas tablas.|
|left|Devuelve todas las filas de la izquierda y las coincidencias de la derecha. Si no hay coincidencia, los valores de la derecha son null.|
|right|Igual que left, pero para el DataFrame derecho.|
|outer|Devuelve todas las filas, con null donde no hay coincidencia.|
|semi|Devuelve solo las filas del DataFrame izquierdo que tienen coincidencia, sin añadir columnas del derecho.|
|anti|Devuelve las filas del DataFrame izquierdo que no tienen coincidencia en el derecho.|

In [ ]:
# Añade los títulos de las películas a los datos de puntuaciones
ratings_with_titles = ratings.join(movies.select("movieId", "title"), on="movieId", how="inner")
ratings_with_titles.show(10)

## Aggregate

Las agregaciones te permiten resumir o condensar la información de un DataFrame usando funciones como avg, count, max, min, sum, etc.

In [ ]:
# Calcula el Top 5 de películas con mayor promedio de rating
movie_stats = ratings_with_titles.groupBy("title") \
    .agg(f.count("rating").alias("num_ratings"), f.avg("rating").alias("avg_rating")) \
    .orderBy("avg_rating", ascending=False)

print('Top 5 de películas con mayor media de valoraciones:')
movie_stats.show(5, truncate=False)

## Función Window

Las window functions permiten realizar cálculos sobre un grupo de filas relacionadas, pero sin colapsarlas en una sola fila (a diferencia de groupBy). Muy útil para rankings, medias móviles, diferencias entre filas, etc.

#### Componentes de una Window

- *partitionBy*: cómo dividir los datos (ej. por usuario)
- *orderBy*: cómo ordenar cada partición (ej. por valoraciones)

In [ ]:
#Película más valorada por cada usuario
window_user = Window.partitionBy("userId").orderBy(ratings_with_titles["rating"].desc())

top_movies_per_user = ratings_with_titles \
    .withColumn("rank", f.row_number().over(window_user)) \
    .filter("rank = 1")

print('Película más valorada por cada usuario:')
top_movies_per_user.select("userId", "title", "rating").show(10)

In [24]:
spark.stop()

## Conclusión

| Concepto             | ¿Para qué se usa?                           |
| -------------------- | ------------------------------------------- |
| **Joins**            | Combinar datos de distintas fuentes         |
| **Aggregations**     | Resumir información (medias, conteos, etc.) |
| **Window Functions** | Cálculos avanzados sobre grupos sin agrupar |